In [1]:
import os
import pandas as pd
import numpy as np
from tqdm import tqdm

In [65]:
path = "F:\Pfactor_clean"
file_name_list = os.listdir(path)
for year in ['2015', '2016', '2017', '2018', '2019', '2020', '2021', '2022', '2023']:
    factor_concat = {}
    pbar = tqdm(file_name_list)
    for file_name in pbar:
        pbar.set_description(f"{year}")
        factor_name = file_name[:-8]
        factor = pd.read_feather(f"{path}\\{file_name}").set_index('dt')
        factor = factor.loc[f"{year}-01-01":f"{year}-12-31", :]
        factor_concat[factor_name] = factor
    factor_concat = pd.concat(factor_concat, axis=1)
    factor_concat.to_pickle(f"./data/factor_concat_{year}.pkl")
del factor_concat

2023: 100%|██████████| 982/982 [04:25<00:00,  3.70it/s]


In [66]:
quantile = 2
stock_return = pd.read_pickle("./data/stock_return.pkl")
for year in ['2015', '2016', '2017', '2018', '2019', '2020', '2021', '2022', '2023']:
    print(year)
    factor_stack = pd.read_pickle(f"./data/factor_concat_{year}.pkl").stack()
    quantile_return = stock_return.stack()
    quantile_return = quantile_return.groupby('dt').apply(
        lambda x: pd.qcut(x, np.arange(quantile + 1) / quantile, np.arange(quantile))
    )
    common_idx = np.intersect1d(factor_stack.index, quantile_return.index)
    factor_stack = factor_stack.loc[common_idx, :]
    quantile_return = quantile_return[common_idx]
    factor_stack.to_pickle(f"./data/processed_data/factor_stack_{year}.pkl")
    quantile_return.to_pickle(f"./data/processed_data/quantile_return_{year}.pkl")

2015
2016
2017
2018
2019
2020
2021
2022
2023


In [153]:
# factor_stack = pd.read_pickle(f"./data/processed_data/factor_stack_2023.pkl")
code_num_dict = pd.read_pickle(f"./data/processed_data/code_num_dict.pkl")
factor_stack['code'] = factor_stack.reset_index()['code'].str.split('.', expand=True)[1].values
factor_stack.iloc[:,982]

dt          code     
2023-01-03  000001.SZ    SZ
            000002.SZ    SZ
            000006.SZ    SZ
            000007.SZ    SZ
            000008.SZ    SZ
                         ..
2023-03-29  688798.SH    SH
            688799.SH    SH
            688800.SH    SH
            688819.SH    SH
            688981.SH    SH
Name: code, Length: 252265, dtype: object

In [48]:
stock_return = pd.read_pickle("./data/stock_return.pkl")
stock_return.tail()

,000001.SZ,000002.SZ,000004.SZ,000005.SZ,000006.SZ,000007.SZ,000008.SZ,000009.SZ,000010.SZ,000011.SZ,...,688786.SH,688787.SH,688788.SH,688789.SH,688793.SH,688798.SH,688799.SH,688800.SH,688819.SH,688981.SH
dt,,,,,,,,,,,,,,,,,,,,,
2023-03-27,-0.001581,-0.001949,0.011051,-0.010791,-0.039486,-0.021656,-0.008148,-0.010336,-0.041426,-0.017294,...,-0.038314,0.114078,-0.004125,-0.067471,-0.053966,-0.006448,0.014540,-0.023818,0.012867,0.090452
2023-03-28,-0.010211,-0.017520,-0.022167,-0.011453,-0.040195,-0.011728,0.008215,0.000889,-0.045917,-0.017806,...,-0.038921,0.007145,0.012863,-0.022989,-0.041505,0.117735,-0.003170,-0.036197,-0.003668,0.168937
2023-03-29,0.001595,-0.007793,-0.019236,-0.005518,-0.009866,-0.023299,0.008510,0.010705,-0.077223,-0.008732,...,-0.056249,-0.168016,0.142500,-0.035468,0.008901,0.137796,-0.001567,-0.003536,-0.013972,0.220522
2023-03-30,-0.003942,0.009618,0.025745,0.005549,0.019821,-0.011835,0.025553,0.003452,-0.039399,0.019888,...,-0.042899,-0.009663,0.130799,-0.045632,-0.014048,0.138248,0.032687,0.041307,-0.023289,0.228106
2023-03-31,0.011952,0.023620,-0.016041,-0.010976,-0.009807,-0.023513,-0.004331,0.000888,-0.089168,0.012930,...,-0.022401,-0.154377,0.086582,-0.045515,-0.047043,0.123679,0.049176,0.053028,-0.021342,0.192132


In [5]:
quantile = 2
quantile_return = stock_return.stack()
quantile_return = quantile_return.groupby('dt').apply(
    lambda x: pd.qcut(x, np.arange(quantile + 1) / quantile, np.arange(quantile))
)
quantile_return.head()

dt          code     
2014-01-02  000001.SZ    1
            000002.SZ    0
            000004.SZ    1
            000005.SZ    1
            000006.SZ    0
dtype: category
Categories (2, int64): [0 < 1]

In [6]:
common_idx = np.intersect1d(factor_stack.index, quantile_return.index)
factor_stack = factor_stack.loc[common_idx, :]
quantile_return = quantile_return[common_idx]

In [128]:
from pytorch_tabnet.tab_model import TabNetClassifier

model = TabNetClassifier(device_name='cuda')
model.load_model("./log/tabnet/basic/2018.zip")
model.input_dim

E:\Anaconda\lib\site-packages\pytorch_tabnet\abstract_model.py:75: UserWarning: Device used : cuda
  warnings.warn(f"Device used : {self.device}")


982

In [165]:
industry_info = pd.read_pickle("F:\Trade_data\\industry_info.pkl").stack()
industry_list = np.unique(industry_info.values).tolist()
industry_info = industry_info.replace(industry_list, [i for i in range(len(industry_list))])
industry_info = industry_info.loc[factor_stack.index]
industry_info

dt                   
2023-01-03  000001.SZ    34
            000002.SZ    15
            000006.SZ    15
            000007.SZ     8
            000008.SZ    17
                         ..
2023-03-29  688798.SH    22
            688799.SH     7
            688800.SH    22
            688819.SH    21
            688981.SH    22
Length: 252265, dtype: int64

In [26]:
path = "F:\Pfactor_evaluation"
file_name_list = os.listdir(path)
ic_mean = []
for file_name in tqdm(file_name_list):
    factor_name = file_name[:-4]
    evaluation = pd.read_pickle(os.path.join(path,file_name))
    ic_series = evaluation['IC_report']['IC_series'].to_frame(factor_name)
    ic_series['year'] = ic_series.index.year
    ic_series['month'] = ic_series.index.month
    ic_series = ic_series.groupby(['year', 'month']).mean().rolling(36).mean()
    ic_series = ic_series.loc[(slice(2018,2023), slice(None))]
    ic_series = abs(ic_series)
    ic_mean.append(ic_series)

ic_mean = pd.concat(ic_mean, axis=1)
ic_mean

100%|██████████| 982/982 [00:02<00:00, 428.71it/s]


alphaP0001  alphaP0002  alphaP0003  alphaP0004  alphaP0005  \
year month                                                               
2018 1        0.064256    0.037538    0.018742    0.072682    0.049580   
     2        0.062380    0.038277    0.013857    0.069997    0.049450   
     3        0.060962    0.038166    0.011515    0.069898    0.052056   
     4        0.059174    0.035061    0.009746    0.069227    0.054544   
     5        0.058925    0.035640    0.011139    0.068624    0.055583   
...                ...         ...         ...         ...         ...   
2022 11       0.051035    0.025300    0.021771    0.058976    0.038406   
     12       0.050948    0.024856    0.021813    0.059817    0.039552   
2023 1        0.054455    0.032431    0.024260    0.062093    0.038174   
     2        0.056463    0.033001    0.026061    0.065433    0.039356   
     3        0.055577    0.031393    0.027194    0.065202    0.037612   

            alphaP0006  alphaP0007  alphaP0008  alphaP0009  alphaP0010  ...  \
year month                                                              ...   
2018 1        0.034902    0.008648    0.029686    0.009959    0.015693  ...   
     2        0.034938    0.007318    0.028101    0.009193    0.014881  ...   
     3        0.034868    0.006738    0.027658    0.008879    0.014649  ...   
     4        0.035721    0.006730    0.028056    0.008668    0.014942  ...   
     5        0.036069    0.006416    0.027459    0.007964    0.013736  ...   
...                ...         ...         ...         ...         ...  ...   
2022 11       0.021342    0.012093    0.012506    0.005012    0.008932  ...   
     12       0.021065    0.013639    0.012350    0.004013    0.007627  ...   
2023 1        0.021398    0.012707    0.012640    0.004223    0.007189  ...   
     2        0.021299    0.015056    0.012010    0.004073    0.007383  ...   
     3        0.020963    0.013343    0.012432    0.003373    0.007578  ...   

            alphaP2019_5_20  alphaP2019_5_200  alphaP2019_5_5  \
year month                                                      
2018 1             0.030841          0.000271        0.020572   
     2             0.026207          0.002424        0.018642   
     3             0.023812          0.003289        0.016984   
     4             0.022525          0.000951        0.016340   
     5             0.021724          0.000020        0.015943   
...                     ...               ...             ...   
2022 11            0.037388          0.021298        0.033233   
     12            0.036506          0.021494        0.033047   
2023 1             0.039008          0.017912        0.034124   
     2             0.043013          0.023147        0.036469   
     3             0.045333          0.026712        0.036982   

            alphaP2019_5_60  alphaP2019_60_10  alphaP2019_60_120  \
year month                                                         
2018 1             0.009006          0.010486           0.003807   
     2             0.004704          0.006605           0.000227   
     3             0.002787          0.004544           0.001523   
     4             0.002663          0.003622           0.000164   
     5             0.004842          0.003143           0.002161   
...                     ...               ...                ...   
2022 11            0.022458          0.023974           0.019504   
     12            0.022532          0.023392           0.019990   
2023 1             0.023494          0.022093           0.017579   
     2             0.027554          0.025456           0.021667   
     3             0.029158          0.028041           0.024625   

            alphaP2019_60_20  alphaP2019_60_200  alphaP2019_60_5  \
year month                                                         
2018 1              0.024933           0.000149         0.033247   
     2              0.020506           0.002858         0.037016   
     3            

In [60]:
selected_factor_dict = {}
for (year, month) in ic_mean.index:
    selected_factor = ic_mean.loc[(year, month)]
    selected_factor = selected_factor[selected_factor>0.01].index.tolist()
    selected_factor = np.intersect1d(factor_stack.columns, selected_factor)
    if month<10:
        selected_factor_dict[f"{year}_0{month}"] = selected_factor
    else:
        selected_factor_dict[f"{year}_{month}"] = selected_factor
pd.to_pickle(selected_factor_dict, "./data/selected_factor_dict.pkl")

In [9]:
for year in ['2015', '2016', '2017', '2018', '2019', '2020', '2021', '2022', '2023']:
    factor_stack = pd.read_pickle(f"./data/processed_data/factor_stack_{year}.pkl")
    pbar = tqdm(["01", "02", "03", "04", "05", "06", "07", "08", "09", "10", "11", "12"])
    for month in pbar:
        pbar.set_description(f"{year}-{month}")
        if month == "02":
            if year in ["2016", "2020"]:
                day = "29"
            else:
                day = "28"
        elif month in ["01", "03", "05", "07", "08", "010", "012"]:
            day = "31"
        else:
            day = "30"
        temp_factor_stack = factor_stack.loc[(slice(f"{year}-{month}-01", f"{year}-{month}-{day}"), slice(None)), :]
        temp_factor_stack.to_pickle(f"./data/processed_data/factor_stack_{year}_{month}.pkl")
del temp_factor_stack

2023-12: 100%|██████████| 12/12 [00:01<00:00,  7.07it/s]


In [13]:
for year in ['2015', '2016', '2017', '2018', '2019', '2020', '2021', '2022', '2023']:
    quantile_return = pd.read_pickle(f"./data/processed_data/quantile_return_{year}.pkl")
    pbar = tqdm(["01", "02", "03", "04", "05", "06", "07", "08", "09", "10", "11", "12"])
    for month in pbar:
        pbar.set_description(f"{year}-{month}")
        if month == "02":
            if year in ["2016", "2020"]:
                day = "29"
            else:
                day = "28"
        elif month in ["01", "03", "05", "07", "08", "010", "012"]:
            day = "31"
        else:
            day = "30"
        temp_quantile_return = quantile_return.loc[(slice(f"{year}-{month}-01", f"{year}-{month}-{day}"), slice(None))]
        temp_quantile_return.to_pickle(f"./data/processed_data/by_month/quantile_return_{year}_{month}.pkl")
del temp_quantile_return

2023-12: 100%|██████████| 12/12 [00:00<00:00, 204.04it/s]


In [72]:
from pytorch_tabnet.tab_model import TabNetClassifier
selected_factor_dict = pd.read_pickle("./data/selected_factor_dict.pkl")
year_list = ['2018', '2019', '2020', '2021', '2022', '2023']
month_list = ["01", "02", "03", "04", "05", "06", "07", "08", "09", "10", "11", "12"]
period_list = [f"{year}_{month}" for year in year_list for month in month_list][:-9]
score_list = []
pbar = tqdm(range(len(period_list)))
model = TabNetClassifier(verbose=0)
for i in pbar:
    period = period_list[i]
    pbar.set_description(f"{period}")
    x_test = pd.read_pickle(f"./data/processed_data/by_month/factor_stack_{period}.pkl")
    lookback_list = period_list[:i+1][::-1][:min(10, i+1)]
    score_concat = {}
    for lookback_period in lookback_list:
        selected_factor = selected_factor_dict[lookback_period]
        temp_x_test = x_test[selected_factor]
        model.load_model(f"./log/tabnet/by_month/{lookback_period}.zip")
        score = model.predict_proba(temp_x_test.values)
        score = pd.DataFrame(score[:, 1], index=pd.MultiIndex.from_tuples(x_test.index))
        score.index.names = ["dt", "code"]
        score = score.unstack()
        score.columns = score.columns.droplevel(0)
        score_concat[lookback_period] = score
    score_concat = pd.concat(score_concat, axis=1)
    score_concat.columns.names = ['period', 'code']
    score_concat = score_concat.groupby('code', axis=1).mean()
    score_list.append(score_concat)
score = pd.concat(score_list, axis=1)
score

2018_01:   0%|          | 0/63 [00:00<?, ?it/s]E:\Anaconda\lib\site-packages\pytorch_tabnet\abstract_model.py:75: UserWarning: Device used : cuda
  warnings.warn(f"Device used : {self.device}")
2018_02:   2%|▏         | 1/63 [00:01<01:30,  1.46s/it]E:\Anaconda\lib\site-packages\pytorch_tabnet\abstract_model.py:75: UserWarning: Device used : cuda
  warnings.warn(f"Device used : {self.device}")
E:\Anaconda\lib\site-packages\pytorch_tabnet\abstract_model.py:75: UserWarning: Device used : cuda
  warnings.warn(f"Device used : {self.device}")
2018_03:   3%|▎         | 2/63 [00:03<01:32,  1.52s/it]E:\Anaconda\lib\site-packages\pytorch_tabnet\abstract_model.py:75: UserWarning: Device used : cuda
  warnings.warn(f"Device used : {self.device}")
E:\Anaconda\lib\site-packages\pytorch_tabnet\abstract_model.py:75: UserWarning: Device used : cuda
  warnings.warn(f"Device used : {self.device}")
E:\Anaconda\lib\site-packages\pytorch_tabnet\abstract_model.py:75: UserWarning: Device used : cuda
  warning

code,000001.SZ,000002.SZ,000004.SZ,000005.SZ,000008.SZ,000009.SZ,000010.SZ,000011.SZ,000012.SZ,000014.SZ,...,688786.SH,688787.SH,688788.SH,688789.SH,688793.SH,688798.SH,688799.SH,688800.SH,688819.SH,688981.SH
dt,,,,,,,,,,,,,,,,,,,,,
2018-01-02,0.500000,0.437432,0.516060,0.500000,0.516266,0.500000,0.454898,0.477160,0.305942,0.500000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2018-01-03,0.448776,0.456896,0.411087,0.473466,0.550808,0.507384,0.374170,0.500000,0.339794,0.500000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2018-01-04,0.491105,0.430737,0.343779,0.500000,0.571447,0.478382,0.460320,0.494124,0.394796,0.488018,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2018-01-05,0.500000,0.472005,0.422822,0.488763,0.567586,0.515146,0.357862,0.476069,0.479109,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2018-01-08,0.486296,0.499377,0.406767,0.486811,0.573538,0.557276,NaN,0.456051,0.453281,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-03-23,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.498235,0.381086,0.499364,0.503538,0.462036,0.412930,0.494599,0.431385,0.491211,0.407250
2023-03-24,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.500426,0.390100,0.518553,0.517665,0.462307,0.380941,0.494219,0.407509,0.496375,0.434147
2023-03-27,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.501401,0.368752,0.519412,0.468623,0.460179,0.391754,0.498617,0.429173,0.523017,0.449001


In [73]:
score = pd.concat(score_list)
score.to_pickle(f"./data/score/mean_score_select.pkl")

In [67]:
factor_stack.columns

Index(['alphaP0001', 'alphaP0002', 'alphaP0003', 'alphaP0004', 'alphaP0005',
       'alphaP0006', 'alphaP0007', 'alphaP0008', 'alphaP0009', 'alphaP0010',
       ...
       'alphaP2019_5_20', 'alphaP2019_5_200', 'alphaP2019_5_5',
       'alphaP2019_5_60', 'alphaP2019_60_10', 'alphaP2019_60_120',
       'alphaP2019_60_20', 'alphaP2019_60_200', 'alphaP2019_60_5',
       'alphaP2019_60_60'],
      dtype='object', length=982)